In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# Define the CNN model for image feature extraction
base_model = ResNet50(weights='imagenet', include_top=False)
image_input = Input(shape=(224, 224, 3))
x = base_model(image_input)
x = layers.GlobalAveragePooling2D()(x)
image_features = layers.Dense(256, activation='relu')(x)
image_features_output = layers.RepeatVector(40)(image_features)

# Define the RNN model for caption generation
caption_input = Input(shape=(40,))
caption_embedding = layers.Embedding(vocab_size, embedding_dim, input_length=40)(caption_input)
caption_features = layers.LSTM(256)(caption_embedding)
caption_features_output = layers.RepeatVector(1)(caption_features)

# Concatenate image features and caption features
decoder_input = layers.concatenate([image_features_output, caption_features_output], axis=1)

# LSTM decoder
decoder_lstm = layers.LSTM(512, return_sequences=True)(decoder_input)
output = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax'))(decoder_lstm)

# Define the model
model = Model(inputs=[image_input, caption_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model
model.fit([images, captions_in], captions_out, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)

# Generate captions for new images
def generate_caption(image):
    image_feature = base_model.predict(image)
    caption = ['<start>']
    for _ in range(max_caption_length):
        sequence = tokenizer.texts_to_sequences([caption])[0]
        sequence = pad_sequences([sequence], maxlen=max_caption_length)
        prediction = model.predict([image_feature, sequence], verbose=0)
        predicted_word_index = np.argmax(prediction)
        predicted_word = index_to_word[predicted_word_index]
        if predicted_word == '<end>':
            break
        caption.append(predicted_word)
    return ' '.join(caption[1:-1])  # Remove <start> and <end> tokens

# Example usage
image = load_and_preprocess_image('path_to_image.jpg')
caption = generate_caption(image)
print("Generated Caption:", caption)


NameError: name 'vocab_size' is not defined